# Application Monitors with Watson OpenScale

This notebook should be run in a Watson Studio project, using Default Python 3.6 runtime environment. It requires service credentials and a Cloud API key to access the following Cloud services:
* Watson OpenScale

The notebook will configure monitors in OpenScale for the German Credit Risk model. The notebook assumes the model has been created/deployed to Watson Machine Learning and that the subscription has been created in Watson OpenScale.

#### Dependency Setup

In [ ]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

#### Configure Service Credentials

Update the two cells below with your Cloud API Key and your Watson Machine Learning service credentials.

In [ ]:
CLOUD_API_KEY = "PASTE HERE"

#### Model Parameters

__Ensure that the two parameters match the model / deployment you have previously subscribed__

In [ ]:
MODEL_NAME = "Spark German Risk Model"
DEPLOYMENT_NAME = "Spark German Risk Deployment"

### Application Monitors

#### Get Watson OpenScale GUID
Each instance of OpenScale has a unique ID. We can get this value using the Cloud API key specified at the beginning of the notebook.

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.utils import get_instance_guid

wos_client = None
WOS_GUID = get_instance_guid(api_key=CLOUD_API_KEY)
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print("Watson OpenScale GUID: {}".format(WOS_GUID))

wos_client = APIClient(aios_credentials=WOS_CREDENTIALS)
print("Watson OpenScale Python Client Version: {}".format(wos_client.version))

#### Get subscription

We have previously subscribed Watson OpenScale to our machine learning model. Here we get that subscription.

In [ ]:
wos_client.data_mart.subscriptions.list()

subscriptions_uids = wos_client.data_mart.subscriptions.get_uids()
subscription_id = None
for sub in subscriptions_uids:
    if wos_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
        subscription = wos_client.data_mart.subscriptions.get(sub)
        subscription_id = sub
        break
            
if subscription is None:
    print('Subscription not found.')
    
print("Subscription ID: {}".format(subscription_id))

data_mart_id = subscription.get_details()['metadata']['url'].split('/service_bindings')[0].split('marts/')[1]
print("Data Mart ID: {}".format(data_mart_id))

business_application_url = "/".join((WOS_CREDENTIALS['url'], data_mart_id,"v2", "business_applications" ))
print("Business Application URL: {}".format(business_application_url))

monitor_instances_url = "/".join((WOS_CREDENTIALS['url'], data_mart_id,"v2", "monitor_instances" ))
print("Monitor Instances URL: {}".format(monitor_instances_url))
      

#### Enable Business Application Monitor


In [ ]:
import requests
import time

payload_business_app = {
    "name": "Credit Risk Application",
    "description": "Test Business Application",
    "payload_fields": [
        {
            "name": "LoanDuration",
            "type": "number",
        },
        {
            "name": "LoanPurpose",
            "type": "string",
        },
        {
            "name": "LoanAmount",
            "type": "number",
        },
        {
            "name": "InstallmentPercent",
            "type": "number",
        },
        {
            "name": "AcceptedPercent",
            "type": "number",
        },
         {
            "name": "Accepted",
            "type": "number",
        },
        {
            "name": "AmountGranted",
            "type": "number",
        },
        {
            "name": "transaction_id",
            "type": "string",
        }     
    ],
    "business_metrics": [
        {
            "name": "Accepted Credits",
            "expected_direction": "increasing",
            "thresholds": [
                {
                    "type": "lower_limit",
                    "default": 0,
                }
            ],
            "required": False,
            "calculation_metadata": {
                "field_name": "Accepted",
                "aggregation": "sum",
                "time_frame": {
                    "count": 1,
                    "unit": "day"
                }
            }
        },
        {
            "name": "Credit Amount Granted",
            "expected_direction": "increasing",
            "thresholds": [
                {
                    "type": "lower_limit",
                    "default": 10000,
                }
            ],
            "required": False,
            "calculation_metadata": {
                "field_name": "AmountGranted",
                "aggregation": "sum",
                "time_frame": {
                    "count": 1,
                    "unit": "day"
                }
            }
        }
    ],
    "subscription_ids": subscriptions_uids
}

response = requests.post(url=business_application_url, headers=wos_client._get_headers(), json=payload_business_app)
business_application_id = response.json()['metadata']['id']
time.sleep(10)

In [ ]:
business_application_details_url = "/".join((business_application_url, business_application_id))
print(business_application_details_url)

business_app_status = None
while business_app_status != 'active':
    business_application_details = requests.get(url = business_application_details_url, headers=wos_client._get_headers()).json()
    business_app_status = business_application_details['entity']['status']['state']
    if business_app_status != 'active':
        print(datetime.utcnow().strftime('%H:%M:%S'), business_app_status)
        time.sleep(10)

print("Business Application Status: {}".format(business_app_status))

In [ ]:
business_payload_data_set_id = business_application_details['entity']['business_payload_data_set_id']
business_metrics_monitor_instance_id =  business_application_details['entity']['business_metrics_monitor_instance_id']

In [ ]:
response = requests.get(url=monitor_instances_url, headers=wos_client._get_headers())
instances = response.json()['monitor_instances']

for instance in instances:
    if 'managed_by' in instance['entity'] and instance['entity']['managed_by'] == business_application_id:
        if instance['entity']['monitor_definition_id'] == 'correlations':
            corr_monitor_instance_id = instance['metadata']['id']

    if instance['entity']['monitor_definition_id'] == 'drift':
        drift_instance_id = instance['metadata']['id']

### Insert historical business payload


In [ ]:
import time
import os
from IPython.utils import io
from ibm_ai_openscale.utils.inject_demo_data import DemoData

historicalData = DemoData(aios_credentials=WOS_CREDENTIALS)
historical_data_path=os.getcwd()

with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_business_payloads_week.csv -O history_business_payloads_week.csv
!ls -lh history_business_payloads_week.csv

historicalData.load_historical_business_payload(business_payload_data_set_id, file_path=historical_data_path, file_name="history_business_payloads_week.csv")
time.sleep(60)

### Insert historical BPKIs

In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_business_metrics.json -O history_business_metrics.json
!ls -lh history_business_metrics.json

historicalData.load_historical_kpi_measurements(monitor_instance_id=business_metrics_monitor_instance_id, filename="history_business_metrics.json", file_path=historical_data_path)

### Insert historical drift measurements


In [ ]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_metrics.json -O history_drift_metrics.json
!ls -lh history_drift_metrics.json

historicalData.load_historical_drift_measurements(business_application_id=business_application_id, monitor_instance_id=drift_instance_id, filename="history_drift_metrics.json", file_path=historical_data_path)

### Run Correlation Monitor


In [ ]:
payload = {
            "triggered_by": "user",
            "parameters": {
                "max_number_of_days": "1000"
            },
            "business_metric_context": {
                "business_application_id": business_application_id,
                "metric_id": "",
                "transaction_data_set_id": "",
                "transaction_batch_id": ""
            }
        }

response = requests.post(url = "/".join((monitor_instances_url, corr_monitor_instance_id,"runs")), json=payload, headers=wos_client._get_headers())

In [ ]:
corr_status = None
while corr_status != 'active':
    corr_details = requests.get(url = "/".join((monitor_instances_url, corr_monitor_instance_id)), headers=wos_client._get_headers()).json()
    corr_status = corr_details['entity']['status']['state']
    if corr_status != 'active':
        print(datetime.utcnow().strftime('%H:%M:%S'), corr_status)
        time.sleep(10)
print("Correlation status: {}".format(corr_status))

## Next steps


__Return to the workshop instruction book.__


## Credits

This notebook was adapted from the following sources:

* [Monitor Models Code Pattern](https://github.com/IBM/monitor-wml-model-with-watson-openscale)
* [OpenScale Labs](https://github.com/pmservice/OpenScale-Labs)
* [OpenScale Tutorials](https://github.com/pmservice/ai-openscale-tutorials)

#### Original Authors
* Eric Martens, is a technical specialist having expertise in analysis and description of business processes, and their translation into functional and non-functional IT requirements. He acts as the interpreter between the worlds of IT and business.
* Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.
